## 0. 라이브러리 다운로드

In [2]:
# !pip install torch transformers

   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 1.6/203.0 MB 7.6 MB/s eta 0:00:27
    --------------------------------------- 3.4/203.0 MB 7.7 MB/s eta 0:00:26
    --------------------------------------- 4.2/203.0 MB 7.6 MB/s eta 0:00:27
   - -------------------------------------- 5.5/203.0 MB 6.5 MB/s eta 0:00:31
   -- ------------------------------------- 11.8/203.0 MB 11.4 MB/s eta 0:00:17
   --- ------------------------------------ 19.4/203.0 MB 15.5 MB/s eta 0:00:12
   ----- ---------------------------------- 29.1/203.0 MB 19.8 MB/s eta 0:00:09
   ------ --------------------------------- 34.1/203.0 MB 20.6 MB/s eta 0:00:09
   -------- ------------------------------- 42.7/203.0 MB 22.8 MB/s eta 0:00:08
   ---------- ----------------------------- 54.3/203.0 MB 26.0 MB/s eta 0:00:06
   ------------ --------------------------- 61.3/203.0 MB 26.8 MB/s eta 0:00:06
   -------------- ------------------------- 73.1/203.0 MB

In [5]:
# !pip install accelerate

In [ ]:
# !python -m pip install librosa soundfile langchain sentence-transformers

In [19]:
!pip install langchain sentence-transformers

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 23.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 36.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 58.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 50.3 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using 

In [ ]:
!pip install accelerate

## 1. 모듈 불러오기

In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

c:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.1. 모델 불러오기 및 파이프라인 설정

In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype,
    # low_cpu_mem_usage=True, 
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cpu


In [14]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

c:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dm705\.cache\huggingface\hub\datasets--distil-whisper--librispeech_long. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating validation split: 100%|██████████| 1/1 [00:00<00:00, 13.21 examples/s

In [21]:
!pip install numpy==1.23.4

     ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
     ------------- -------------------------- 3.7/10.7 MB 18.2 MB/s eta 0:00:01
     --------------------------------------- 10.7/10.7 MB 27.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "C:\Users\dm705\AppData\Local\Programs\Python\Python312\Lib\si

In [17]:
import librosa

In [25]:
import numpy as np

np.__version__

'2.2.1'

In [20]:
sample = dataset[0]["audio"]

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'